# Three ways for geometric modeling in Netgen

## Spline geometry

This is the oldest way, now made compatible with webgui.


In [ ]:
import ngsolve as ng
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry

def CreateGeom():
    geo = SplineGeometry()

    #    3-------2------5
    #    |       |      |
    #    |   1   |      |
    #    |       |  2   |
    #    |       |      |
    #    0-------1------4

    #        0        1        2        3       4      5
    pts = [(-3, 0), (0, 0), (0, 2), (-3, 2), (3, 0), (3, 2)]
    pn = [geo.AppendPoint(*p) for p in pts]

    #     from, to, name,  left, right
    lns = [(0,  1,  'bot', 1,    0), 
           (1,  2,  'mid', 1,    2), 
           (2,  3,  'top', 1,    0),
           (3,  0,  'lft', 1,    0),
           (1,  4,  'bot', 2,    0),
           (4,  5,  'rgt', 2,    0),
           (5,  2,  'top', 2,    0)
          ]
    for frm, to, name, left, right in lns:
        geo.Append(["line", pn[frm], pn[to]], 
                   bc=name, leftdomain=left, rightdomain=right)
    geo.SetMaterial(1, "lftbar")
    geo.SetMaterial(2, "rgtbar")
    return geo

In [ ]:
mesh = ng.Mesh(CreateGeom().GenerateMesh(maxh=1/4))
Draw(mesh);

Often we need to know the orientation of the normal vector on the boundaries. Below, we check that the normal vector obtained from NGSolve's `specialcf.normal` is the unit outward pointing normal on the  boundaries of the domains  created as `SplineGeometry` objects.

In [ ]:
def norientation(mesh):
    """Use a boundary interpolation to view the normal orientation"""
    R = ng.HDiv(mesh, order=5, RT=True, dirichlet='.*')
    r = ng.GridFunction(R)
    n = ng.specialcf.normal(mesh.dim)
    r.Set(n, ng.BND)
    Draw(r, mesh, vectors={'grid_size': 50});

In [ ]:
norientation(mesh)

## Constructive Solid Geometry (CSG)

CSG models can be built in both 2D and 3D. See the NGSolve documentation for all the available primitives. Here is a 2D example.


In [ ]:
import ngsolve as ng
from ngsolve.webgui import Draw
from netgen.geom2d import CSG2d, Rectangle

def CreateGeomCSG():
    geo = CSG2d()
    Rlft = Rectangle(pmin=(-3, 0), pmax=(0, 2), mat='lftbar', 
                     bottom='bot', top='top', left='lft', right='mid')
    Rrgt = Rectangle(pmin=(0, 0),  pmax=(3, 2), mat='rgtbar',
                     bottom='bot', top='top', left='mid', right='rgt')
    geo.Add(Rlft)
    geo.Add(Rrgt)
    return geo

In [ ]:
meshCSG = ng.Mesh(CreateGeomCSG().GenerateMesh(maxh=1/4))
norientation(meshCSG)

Note how the orientation of the normal on the left edge is not outward in this CSG2D object. (The NGSolve development team is aware of this. This may or may not change in the future, so users have to be wary to not make assumptions on the direction of the normal.)

## OpenCascade (OCC)

This is the newest and the most powerful method. It allows one to view geometry objects even before they are meshed. However, unlike the the other two methods, this option requires you to compile NGSolve after installing the free [OpenCascade](https://www.opencascade.com/) software and using it as a dependency.  I encourage you to do so, recompiling your installation if necessary.

In [ ]:
import ngsolve as ng
from ngsolve.webgui import Draw
from netgen.occ import MoveTo, X, Y, Rectangle, Glue, OCCGeometry
from netgen.webgui import Draw as DrawGeo

def CreateGeomOCC():
    rgtbar = Rectangle(3, 2).Face()
    lftbar = MoveTo(-3, 0).Rectangle(3, 2).Face()
    lftbar.faces.name = 'lftbar'
    rgtbar.faces.name = 'rgtbar'
    lftbar.edges.Min(X).name = 'lft'
    lftbar.edges.Min(Y).name = 'bot'
    lftbar.edges.Max(Y).name = 'top'
    rgtbar.edges.Max(X).name = 'rgt'
    rgtbar.edges.Min(Y).name = 'bot'
    rgtbar.edges.Max(Y).name = 'top'
    bar = Glue([lftbar, rgtbar])
    return bar

geo = CreateGeomOCC()
DrawGeo(geo);

In [ ]:
g = OCCGeometry(CreateGeomOCC(), dim=2)
meshOCC = ng.Mesh(g.GenerateMesh(maxh=1/4))
norientation(meshOCC)